In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob as glob
import sys
import os
import time
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
#import mpld3
#mpld3.enable_notebook()
import pickle
import datetime
import socket
import time
import traceback
import yaml
from pathlib import Path

In [ ]:
import logging
logger = logging.getLogger(__name__)

# Configure the logging system
logging.basicConfig(
    level=logging.INFO,  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Format of the log messages
    datefmt='%Y-%m-%d %H:%M:%S',  # Format of the timestamp
)
logger = logging.getLogger(__name__)

In [ ]:

sys.path.append(os.path.abspath(os.path.join('../assetto_corsa_gym')))
from AssettoCorsaEnv.data_loader import *
from AssettoCorsaEnv.motec_loader import MotecLoader

In [ ]:
data_dir = r"C:\Users\aremonda\Dropbox\zshared_temp\assetto_corsa_rl\data_sets"
dataset_path = Path(data_dir + os.sep)

output_path = "output"
os.makedirs(output_path, exist_ok=True)

In [ ]:
def append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list):
    # Create the dictionary
    summary_dict = {
        "car_name": car_name,
        "track_name": track_name,
        "stints": len(all_users),
        "sessions": len(load_paths),
        "number_of_laps": stint_stats.number_of_laps.sum(),
        "total_steps": stint_stats.steps.sum()
    }
    # Append the dictionary to the list
    summary_list.append(summary_dict)

summary_list = []

In [ ]:
car_name = "dallara_f317"
track_name = "ks_barcelona-layout_gp"

load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"})
load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"}, filter_car=car_name, filter_track=track_name)
load_paths

In [ ]:
def lap_times_list(df):
    laps_info = []
    for i in list(set(df["LapCount"])):
        if i == 0: # discard out lap
            continue
        l = df[df["LapCount"] == i]
        t = l["lastLapTime"].values[-1]

        oot_per = l.numberOfTyresOut [l.numberOfTyresOut > 2 ].sum() / len(l) * 100

        #print(i, data_loader.seconds_to_mm_ss_mmm(t), t)
        r = {"LapCount": i, "LapTime": t.round(4), "LapTimeStr": seconds_to_mm_ss_mmm(t),
             "speed_max": l["speed"].max() * 3.6, "speed_avg": l["speed"].mean() * 3.6,
             "steps": len(l),
             "oot_per": oot_per
            }
        laps_info.append(r.copy())
    laps_info = pd.DataFrame(laps_info, index=None).round(3)
    best_lap = laps_info['LapTime'].idxmin()+1

    stint_stats = {
        "name": load_path.split("/")[-3],
        "car": load_path.split("/")[-4],
        "track": load_path.split("/")[-5],
        "path": load_path,
        "best_lap": best_lap,
        "best_lap_time": laps_info.loc[best_lap-1]["LapTime"],
        "best_lap_time_str": laps_info.loc[best_lap-1]["LapTimeStr"],
        "number_of_laps": len(laps_info),
        "speed_max": laps_info["speed_max"].max(),
        "speed_mean": laps_info["speed_avg"].max(),
        "steps": laps_info.steps.sum(),
    }
    laps_info = laps_info[:-1] # remove last lap as it is not complete
    return laps_info, stint_stats

In [ ]:
all_users = []
all_users_laps_info = []

for load_path, _, _ in load_paths:
    laps_path = glob.glob(f'{load_path}/*.ld')
    assert len(laps_path), f"{load_path}, {laps_path}"

    for i, f in enumerate(laps_path):
        logger.info(f)
        lap = MotecLoader(f, ac_fps=50, target_freq=25, ignore_fps_missmatch=True)
        laps_info, stint_stats = lap_times_list(lap.ts)
        all_users.append(stint_stats)
        all_users_laps_info.append(laps_info)

stint_stats = pd.DataFrame(all_users)

all_laps = np.concatenate( [i["LapTime"].values for i in all_users_laps_info] )

In [ ]:
DEFAULT_RC = {
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
	'grid.linestyle': '-',
	'grid.linewidth': 2,
    'grid.color': '#F3F3F3',
	'axes.labelsize': 20,
	'axes.titlesize': 20,
	'axes.facecolor': '#FFF',
	'axes.edgecolor': '#333',
	'legend.fontsize': 18,
	'xtick.labelsize': 16,
	'xtick.bottom': True,
	'xtick.color': '#333',
	'ytick.labelsize': 16,
	'ytick.left': True,
	'ytick.color': '#333',
    'lines.linewidth': 3,
}

COLORS = [
    'tab:red',
	'tab:blue',
	'tab:green',
    'tab:purple',
    'tab:brown',
	'tab:orange',
	'black',
	'tab:cyan',
	'tab:olive',
	'xkcd:gold',
	'xkcd:mint',
    '#abacab',
]

def set_style():
    #matplotlib.use('agg')
    sns.set_context(context='paper', rc=DEFAULT_RC)
    sns.set(style='whitegrid', rc=DEFAULT_RC)
    sns.set_palette(sns.color_palette(COLORS))

import seaborn as sns
set_style()

# Complete updated dictionary
EXPERIMENTS = {
    'HUMAN': {'label': 'Best Human', 'color': 6},
    'SAC-FD': {'label': 'SAC fD', 'color': 0},
    'SAC': {'label': 'SAC', 'color': 1},
    'AC-AI': {'label': 'Built-in AI', 'color': 4},
    'TD-MPC2 FD': {'label': 'TD-MPC2 fD', 'color': 2},
    'TD-MPC2 FT': {'label': 'TD-MPC2 ft', 'color': 3},
    'SAC FD BAD': {'label': 'SAC fD BAD', 'color': 5},
    'SAC FD GOOD': {'label': 'SAC fD GOOD', 'color': 7},
}

def get_color_from_experiment(experiment):
    return COLORS[EXPERIMENTS[experiment]['color']]

def get_label_from_experiment(experiment):
    return EXPERIMENTS[experiment]['label']


In [ ]:
np.max(all_laps)

In [ ]:
# Bin size of 0.1
bin_width = 2
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 146.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color="tab:gray")
#plt.title(f'Distribution of Lap Times. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xlim(96, 146)
plt.xticks(np.arange(96, max_val, 4.))
plt.axvline(x=97.54, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))
plt.axvline(x=99.78, color=get_color_from_experiment("AC-AI"), linestyle='--', linewidth=3, label=get_label_from_experiment("AC-AI"))
plt.axvline(x=98.27, color=get_color_from_experiment("TD-MPC2 FT"), linestyle='--', linewidth=3, label=get_label_from_experiment("TD-MPC2 FT"))


# Adding yellow vertical lines with different labels
yellow_lines = [100.7, 102]  # Example x-values
labels = ['12 hrs', ' 6 hrs']  # Corresponding labels
for x, label in zip(yellow_lines, labels):
    plt.text(x, plt.ylim()[1]*0.75, label, color='black', ha='left', va='bottom', rotation=90, fontsize=16,
             bbox=dict(facecolor='white', edgecolor='none', pad=2))
    if x == 97.52:
        continue
    plt.axvline(x=x, color='gray', linestyle='--', linewidth=2)


plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()




In [ ]:
# Bin size of 0.1
bin_width = 0.1
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 100.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color='tab:gray')
#plt.title(f'Distribution of Lap Times - Zoom-in. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.xlim((97.5, 100))
plt.grid(True)
plt.xticks(np.arange(97.5, max_val, 1.))
plt.axvline(x=97.54, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))
plt.axvline(x=99.78, color=get_color_from_experiment("AC-AI"), linestyle='--', linewidth=3, label=get_label_from_experiment("AC-AI"))
plt.axvline(x=98.27, color=get_color_from_experiment("TD-MPC2 FT"), linestyle='--', linewidth=3, label=get_label_from_experiment("TD-MPC2 FT"))

# Adding yellow vertical lines with different labels
yellow_lines = [97.52, 99.7, 100]  # Example x-values
labels = ['4 days', '30 hrs']  # Corresponding labels
for x, label in zip(yellow_lines, labels):
    plt.text(x, plt.ylim()[1]*0.75, label, color='black', ha='left', va='bottom', rotation=90, fontsize=16,
             bbox=dict(facecolor='white', edgecolor='none', pad=2))
    if x == 97.52:
        continue
    plt.axvline(x=x, color='gray', linestyle='--', linewidth=2)

#plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()

print(f"{car_name} {track_name} stints: {len(all_users)} sessions: {len(load_paths)} number of laps {stint_stats.number_of_laps.sum()} total steps {stint_stats.steps.sum()}")

In [ ]:
append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list)
summary_list

# monza

In [ ]:
car_name = "dallara_f317"
track_name = "monza"

load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"})
load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"}, filter_car=car_name, filter_track=track_name)



In [ ]:
all_users = []
all_users_laps_info = []

for load_path, _, _ in load_paths:
    laps_path = glob.glob(f'{load_path}/*.ld')

    assert len(laps_path), load_path

    for i, f in enumerate(laps_path):
        logger.info(f)
        lap = MotecLoader(f, ac_fps=50, target_freq=25, ignore_fps_missmatch=True)
        laps_info, stint_stats = lap_times_list(lap.ts)
        all_users.append(stint_stats)
        all_users_laps_info.append(laps_info)

stint_stats = pd.DataFrame(all_users)

all_laps = np.concatenate( [i["LapTime"].values for i in all_users_laps_info] )

In [ ]:
# Bin size of 0.1
bin_width = 2
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 150.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color="tab:gray")
#plt.title(f'Distribution of Lap Times. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xlim(102, 146)
plt.xticks(np.arange(102, max_val, 4.))
plt.axvline(x=104.03, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))

# # Adding yellow vertical lines with different labels
# yellow_lines = [100.7, 102]  # Example x-values
# labels = ['12 hrs', ' 6 hrs']  # Corresponding labels
# for x, label in zip(yellow_lines, labels):
#     plt.text(x, plt.ylim()[1]*0.75, label, color='black', ha='left', va='bottom', rotation=90, fontsize=16,
#              bbox=dict(facecolor='white', edgecolor='none', pad=2))
#     if x == 97.52:
#         continue
#     plt.axvline(x=x, color='gray', linestyle='--', linewidth=2)


plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()



In [ ]:
# Bin size of 0.1
bin_width = 0.1
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 106.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color="tab:gray")
#plt.title(f'Distribution of Lap Times. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xlim(104, 106)
plt.xticks(np.arange(104, max_val, 0.5))
plt.axvline(x=104.03, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))

# # Adding yellow vertical lines with different labels
# yellow_lines = [100.7, 102]  # Example x-values
# labels = ['12 hrs', ' 6 hrs']  # Corresponding labels
# for x, label in zip(yellow_lines, labels):
#     plt.text(x, plt.ylim()[1]*0.75, label, color='black', ha='left', va='bottom', rotation=90, fontsize=16,
#              bbox=dict(facecolor='white', edgecolor='none', pad=2))
#     if x == 97.52:
#         continue
#     plt.axvline(x=x, color='gray', linestyle='--', linewidth=2)


plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()



In [ ]:
append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list)
summary_list

In [ ]:
car_name = "dallara_f317"
track_name = "ks_red_bull_ring-layout_gp"

load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"})
load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"}, filter_car=car_name, filter_track=track_name)



In [ ]:
all_users = []
all_users_laps_info = []

for load_path, _, _ in load_paths:
    laps_path = glob.glob(f'{load_path}/*.ld')

    assert len(laps_path), load_path

    for i, f in enumerate(laps_path):
        logger.info(f)
        lap = MotecLoader(f, ac_fps=50, target_freq=25, ignore_fps_missmatch=True)
        laps_info, stint_stats = lap_times_list(lap.ts)
        all_users.append(stint_stats)
        all_users_laps_info.append(laps_info)

stint_stats = pd.DataFrame(all_users)

all_laps = np.concatenate( [i["LapTime"].values for i in all_users_laps_info] )

In [ ]:
np.min(all_laps)

In [ ]:
# Bin size of 0.1
bin_width = 1
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 120.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color="tab:gray")
#plt.title(f'Distribution of Lap Times. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xlim(82, 120)
plt.xticks(np.arange(82, max_val, 2))
plt.axvline(x=83.61, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))
plt.axvline(x=84.85, color=get_color_from_experiment("AC-AI"), linestyle='--', linewidth=3, label=get_label_from_experiment("AC-AI"))

plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()



In [ ]:
# Bin size of 0.1
bin_width = 0.1
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 87.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color="tab:gray")
#plt.title(f'Distribution of Lap Times. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xlim(83, 87)
plt.xticks(np.arange(83, max_val, 1))
plt.axvline(x=83.61, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))
plt.axvline(x=84.85, color=get_color_from_experiment("AC-AI"), linestyle='--', linewidth=3, label=get_label_from_experiment("AC-AI"))

plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()



In [ ]:
append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list)
summary_list

# GT

In [ ]:
car_name = "bmw_z4_gt3"
track_name = "ks_barcelona-layout_gp"

load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"})
load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"}, filter_car=car_name, filter_track=track_name)

all_users = []
all_users_laps_info = []

for load_path, _, _ in load_paths:
    laps_path = glob.glob(f'{load_path}/*.ld')

    assert len(laps_path), load_path

    for i, f in enumerate(laps_path):
        logger.info(f)
        lap = MotecLoader(f, ac_fps=50, target_freq=25, ignore_fps_missmatch=True)
        laps_info, stint_stats = lap_times_list(lap.ts)
        all_users.append(stint_stats)
        all_users_laps_info.append(laps_info)

stint_stats = pd.DataFrame(all_users)

all_laps = np.concatenate( [i["LapTime"].values for i in all_users_laps_info] )

In [ ]:
# Bin size of 0.1
bin_width = 2
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 150
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 6))
plt.hist(all_laps, bins=bins, color='skyblue', edgecolor='black')
plt.title(f'Distribution of Lap Times. Track: {track_name} Car: {car_name}')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xticks(np.arange(108, max_val, 2.))
plt.axvline(x=109.505, color='red', linestyle='--', linewidth=2, label='SAC=109.505@10M steps')
plt.axvline(x=109.655, color='blue', linestyle='--', linewidth=2, label='Built-in AC AI=109.655')
#plt.axvline(x=98.265, color='magenta', linestyle='--', linewidth=2, label='TDMPC=98.265s')


plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.png", bbox_inches='tight', pad_inches=0.1)
plt.show()



In [ ]:
# Bin size of 0.1
bin_width = 2
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 150.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color="tab:gray")
#plt.title(f'Distribution of Lap Times. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xlim(108, max_val)
plt.xticks(np.arange(108, max_val, 2.))
plt.axvline(x=108.96, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))
plt.axvline(x=109.66, color=get_color_from_experiment("AC-AI"), linestyle='--', linewidth=3, label=get_label_from_experiment("AC-AI"))

plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
bin_width = 0.1
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 110.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color='tab:gray')
#plt.title(f'Distribution of Lap Times - Zoom-in. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.xlim((108, max_val))
plt.grid(True)
plt.xticks(np.arange(108, max_val, 0.2))
plt.axvline(x=108.96, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))
plt.axvline(x=109.66, color=get_color_from_experiment("AC-AI"), linestyle='--', linewidth=3, label=get_label_from_experiment("AC-AI"))

#plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()

print(f"{car_name} {track_name} stints: {len(all_users)} sessions: {len(load_paths)} number of laps {stint_stats.number_of_laps.sum()} total steps {stint_stats.steps.sum()}")



In [ ]:
append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list)
summary_list

# Austria

In [ ]:
car_name = "bmw_z4_gt3"
track_name = "ks_red_bull_ring-layout_gp"

load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"})
load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"}, filter_car=car_name, filter_track=track_name)

all_users = []
all_users_laps_info = []

for load_path, _, _ in load_paths:
    laps_path = glob.glob(f'{load_path}/*.ld')

    assert len(laps_path), load_path

    for i, f in enumerate(laps_path):
        logger.info(f)
        lap = MotecLoader(f, ac_fps=50, target_freq=25, ignore_fps_missmatch=True)
        laps_info, stint_stats = lap_times_list(lap.ts)
        all_users.append(stint_stats)
        all_users_laps_info.append(laps_info)

stint_stats = pd.DataFrame(all_users)

all_laps = np.concatenate( [i["LapTime"].values for i in all_users_laps_info] )

In [ ]:
bin_width = 2
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 140.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color="tab:gray")
#plt.title(f'Distribution of Lap Times. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xlim(90, max_val)
plt.xticks(np.arange(90, max_val, 2.))
plt.axvline(x=92.571, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))
plt.axvline(x=91.56, color=get_color_from_experiment("AC-AI"), linestyle='--', linewidth=3, label=get_label_from_experiment("AC-AI"))


plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()



In [ ]:
bin_width = 0.1
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 94.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color="tab:gray")
#plt.title(f'Distribution of Lap Times. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xlim(91, max_val)
plt.xticks(np.arange(91, max_val, 0.2))
plt.axvline(x=92.571, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))
plt.axvline(x=91.56, color=get_color_from_experiment("AC-AI"), linestyle='--', linewidth=3, label=get_label_from_experiment("AC-AI"))


plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list)
summary_list

In [ ]:
# monza

In [ ]:
car_name = "bmw_z4_gt3"
track_name = "monza"

load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"})
load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"}, filter_car=car_name, filter_track=track_name)

all_users = []
all_users_laps_info = []

for load_path, _, _ in load_paths:
    laps_path = glob.glob(f'{load_path}/*.ld')

    assert len(laps_path), load_path

    for i, f in enumerate(laps_path):
        logger.info(f)
        lap = MotecLoader(f, ac_fps=50, target_freq=25, ignore_fps_missmatch=True)
        laps_info, stint_stats = lap_times_list(lap.ts)
        all_users.append(stint_stats)
        all_users_laps_info.append(laps_info)

stint_stats = pd.DataFrame(all_users)

all_laps = np.concatenate( [i["LapTime"].values for i in all_users_laps_info] )

In [ ]:
bin_width = 2
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 150.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color="tab:gray")
#plt.title(f'Distribution of Lap Times. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xlim(110, max_val)
plt.xticks(np.arange(110, max_val, 2.))
plt.axvline(x=111.63, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))
plt.axvline(x=111.44, color=get_color_from_experiment("AC-AI"), linestyle='--', linewidth=3, label=get_label_from_experiment("AC-AI"))

plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()



In [ ]:
bin_width = 0.1
# Calculating number of bins needed
min_val = np.min(all_laps)
max_val = np.max(all_laps)
max_val = 114.
bins = np.arange(min_val, max_val + bin_width, bin_width)

# Plotting the histogram
plt.figure(figsize=(12, 4))
plt.hist(all_laps, bins=bins, edgecolor='black', color="tab:gray")
#plt.title(f'Distribution of Lap Times. Barcelona/F317')
plt.xlabel('Lap time [s]')
plt.ylabel('Number of laps')
plt.grid(True)
plt.xlim(110.5, max_val)
plt.xticks(np.arange(110.5, max_val, 0.5))
plt.axvline(x=111.63, color=get_color_from_experiment("SAC-FD"), linestyle='--', linewidth=3, label=get_label_from_experiment("SAC-FD"))
plt.axvline(x=111.44, color=get_color_from_experiment("AC-AI"), linestyle='--', linewidth=3, label=get_label_from_experiment("AC-AI"))

plt.legend()
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.png", bbox_inches='tight', pad_inches=0.1)
plt.savefig(f"{output_path}/lap_time_distribution_{track_name}_{car_name}_zoom.pdf", bbox_inches='tight', pad_inches=0.1)
plt.show()



In [ ]:
append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list)
summary_list

# miata

In [ ]:
car_name = "dallara_f317"
track_name = "indianapolis_sp"

load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"})
load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"}, filter_car=car_name, filter_track=track_name)

all_users = []
all_users_laps_info = []

for load_path, _, _ in load_paths:
    laps_path = glob.glob(f'{load_path}/*.ld')

    assert len(laps_path), load_path

    for i, f in enumerate(laps_path):
        logger.info(f)
        lap = MotecLoader(f, ac_fps=50, target_freq=25, ignore_fps_missmatch=True)
        laps_info, stint_stats = lap_times_list(lap.ts)
        all_users.append(stint_stats)
        all_users_laps_info.append(laps_info)

stint_stats = pd.DataFrame(all_users)

all_laps = np.concatenate( [i["LapTime"].values for i in all_users_laps_info] )

append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list)


In [ ]:
car_name = "ks_mazda_miata"
track_name = "ks_barcelona-layout_gp"

load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"})
load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"}, filter_car=car_name, filter_track=track_name)

all_users = []
all_users_laps_info = []

for load_path, _, _ in load_paths:
    laps_path = glob.glob(f'{load_path}/*.ld')

    assert len(laps_path), load_path

    for i, f in enumerate(laps_path):
        logger.info(f)
        lap = MotecLoader(f, ac_fps=50, target_freq=25, ignore_fps_missmatch=True)
        laps_info, stint_stats = lap_times_list(lap.ts)
        all_users.append(stint_stats)
        all_users_laps_info.append(laps_info)

stint_stats = pd.DataFrame(all_users)

all_laps = np.concatenate( [i["LapTime"].values for i in all_users_laps_info] )

append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list)

In [ ]:
car_name = "ks_mazda_miata"
track_name = "monza"

load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"})
load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"}, filter_car=car_name, filter_track=track_name)

all_users = []
all_users_laps_info = []

for load_path, _, _ in load_paths:
    laps_path = glob.glob(f'{load_path}/*.ld')

    assert len(laps_path), load_path

    for i, f in enumerate(laps_path):
        logger.info(f)
        lap = MotecLoader(f, ac_fps=50, target_freq=25, ignore_fps_missmatch=True)
        laps_info, stint_stats = lap_times_list(lap.ts)
        all_users.append(stint_stats)
        all_users_laps_info.append(laps_info)

stint_stats = pd.DataFrame(all_users)

all_laps = np.concatenate( [i["LapTime"].values for i in all_users_laps_info] )

append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list)

In [ ]:
car_name = "ks_mazda_miata"
track_name = "ks_red_bull_ring-layout_gp"

load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"})
load_paths = get_all_paths_in_file(dataset_path=dataset_path, file='paths.yml', filter_tags={"type": "human", "data_type": "motec"}, filter_car=car_name, filter_track=track_name)

all_users = []
all_users_laps_info = []

for load_path, _, _ in load_paths:
    laps_path = glob.glob(f'{load_path}/*.ld')

    assert len(laps_path), load_path

    for i, f in enumerate(laps_path):
        logger.info(f)
        lap = MotecLoader(f, ac_fps=50, target_freq=25, ignore_fps_missmatch=True)
        laps_info, stint_stats = lap_times_list(lap.ts)
        all_users.append(stint_stats)
        all_users_laps_info.append(laps_info)

stint_stats = pd.DataFrame(all_users)

all_laps = np.concatenate( [i["LapTime"].values for i in all_users_laps_info] )

append_summary_to_list(car_name, track_name, all_users, load_paths, stint_stats, summary_list)

In [ ]:
pd.DataFrame(summary_list)

In [ ]:
pd.DataFrame(summary_list).to_csv(f"{output_path}/summary.csv", index=False)

In [ ]:
pd.DataFrame(summary_list).sum()